In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from tensorflow.keras import layers, models, optimizers, losses, activations
from keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import StackingClassifier, VotingClassifier
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from scipy.signal import butter, lfilter
import tensorflow as tf
from keras.saving import load_model

from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc

Using [@cdeotte's](https://www.kaggle.com/cdeotte) data generation

In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt

train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
print( train.shape )
display( train.head() )

# CHOICE TO CREATE OR LOAD EEGS FROM NOTEBOOK VERSION 1
CREATE_EEGS = False
TRAIN_MODEL = True

In [ ]:
df = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')
FEATS = df.columns
print(f'There are {len(FEATS)} raw eeg features')
print( list(FEATS) )

In [ ]:
print('We will use the following subset of raw EEG features:')
FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
FEAT2IDX = {x:y for x,y in zip(FEATS,range(len(FEATS)))}
print( list(FEATS) )

In [ ]:
def eeg_from_parquet(parquet_path, display=False):

    # EXTRACT MIDDLE 50 SECONDS
    eeg = pd.read_parquet(parquet_path, columns=FEATS)
    rows = len(eeg)
    offset = (rows-10_000)//2
    eeg = eeg.iloc[offset:offset+10_000]

    if display:
        plt.figure(figsize=(10,5))
        offset = 0

    # CONVERT TO NUMPY
    data = np.zeros((10_000,len(FEATS)))
    for j,col in enumerate(FEATS):

        # FILL NAN
        x = eeg[col].values.astype('float32')
        m = np.nanmean(x)
        if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
        else: x[:] = 0

        data[:,j] = x

        if display:
            if j!=0: offset += x.max()
            plt.plot(range(10_000),x-offset,label=col)
            offset -= x.min()

    if display:
        plt.legend()
        name = parquet_path.split('/')[-1]
        name = name.split('.')[0]
        plt.title(f'EEG {name}',size=16)
        plt.show()

    return data



In [ ]:
%%time

all_eegs = np.load('/kaggle/input/brain-eegs/eegs.npy',allow_pickle=True).item()

In [ ]:
# LOAD TRAIN
EEG_IDS = train.eeg_id.unique()
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

train = df.groupby('eeg_id')[['patient_id']].agg('first')

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
train = train.loc[train.eeg_id.isin(EEG_IDS)]
print('Train Data with unique eeg_id shape:', train.shape )
train.head()

In [ ]:
from scipy.signal import butter, lfilter
def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=7): 
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=False, eegs=all_eegs, mode='train',
                 downsample=5):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.eegs = eegs
        self.mode = mode
        self.downsample = downsample
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = int( np.ceil( len(self.data) / self.batch_size ) )
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)

        return X[:,::self.downsample,:], y
        #return decimate(X, self.downsample, axis=1), y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange( len(self.data) )
        if self.shuffle: np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((len(indexes),10_000,8),dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')

        sample = np.zeros((10_000,X.shape[-1]))
        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            data = self.eegs[row.eeg_id]

            # FEATURE ENGINEER
            sample[:,0] = data[:,FEAT2IDX['Fp1']] - data[:,FEAT2IDX['T3']]
            sample[:,1] = data[:,FEAT2IDX['T3']] - data[:,FEAT2IDX['O1']]

            sample[:,2] = data[:,FEAT2IDX['Fp1']] - data[:,FEAT2IDX['C3']]
            sample[:,3] = data[:,FEAT2IDX['C3']] - data[:,FEAT2IDX['O1']]


            sample[:,4] = data[:,FEAT2IDX['Fp2']] - data[:,FEAT2IDX['C4']]
            sample[:,5] = data[:,FEAT2IDX['C4']] - data[:,FEAT2IDX['O2']]

            sample[:,6] = data[:,FEAT2IDX['Fp2']] - data[:,FEAT2IDX['T4']]
            sample[:,7] = data[:,FEAT2IDX['T4']] - data[:,FEAT2IDX['O2']]



            # STANDARDIZE
            sample = np.clip(sample,-1024,1024)
            sample = np.nan_to_num(sample, nan=0) / 32.0

            # BUTTER LOW-PASS FILTER
            sample = butter_lowpass_filter(sample)

            X[j,] = sample
            if self.mode!='test':
                y[j] = row[TARGETS]
        return X,y

In [ ]:
# USE MIXED PRECISION
MIX = True
if MIX:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
    print('Mixed precision enabled')
else:
    print('Using full precision')

Implementation of ChronoNet architecture

In [ ]:
def chronoNet(input_shape=(2000,8), num_classes=6):
    print(f'Making ChronoNet with input {input_shape} and {num_classes} classes')
    inp = layers.Input(shape=input_shape)
    
    # Inception Style Convolutional layers
    cat = inp
    for i in range(3):
        conv1 = layers.Conv1D(128, 2, strides=2, padding='same')(cat)
        conv2 = layers.Conv1D(128, 4, strides=2, padding='same')(cat)
        conv3 = layers.Conv1D(128, 8, strides=2, padding='same')(cat)
        cat = layers.concatenate([conv1, conv2, conv3])
    
    # DenseNet style RNN layers
    rnn1 = layers.GRU(128, return_sequences=True)(cat)
    rnn2 = layers.GRU(128, return_sequences=True)(rnn1)
    cat = layers.concatenate([rnn1, rnn2])
    for i in range(3):
        rnn = layers.GRU(128, return_sequences=True)(cat)
        cat = layers.concatenate([cat, rnn])

    rnn = layers.GRU(128)(cat)

    x = layers.Dense(6, activation='softmax', dtype='float32')(rnn)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=x)
    loss = tf.keras.losses.KLDivergence()
    opt = tf.keras.optimizers.AdamW(learning_rate = 1e-3, clipvalue=50.0)
    model.compile(loss=loss, 
                  optimizer = opt, 
                  metrics=['accuracy', tf.keras.metrics.AUC()],)
    return model
    
model = chronoNet()
model.summary()
plot_model(model)

In [ ]:
import math
LR_START = 1e-6
LR_MAX = 1e-3
LR_MIN = 1e-6
LR_RAMPUP_EPOCHS = 0
LR_SUSTAIN_EPOCHS = 0
EPOCHS2 = 30

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        decay_total_epochs = EPOCHS2 - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
        decay_epoch_index = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        phase = math.pi * decay_epoch_index / decay_total_epochs
        cosine_decay = 0.5 * (1 + math.cos(phase))
        lr = (LR_MAX - LR_MIN) * cosine_decay + LR_MIN
    return lr

rng = [i for i in range(EPOCHS2)]
lr_y = [lrfn(x) for x in rng]
plt.figure(figsize=(10, 4))
plt.plot(rng, lr_y, '-o')
plt.xlabel('epoch',size=14); plt.ylabel('learning rate',size=14)
plt.title('Cosine Training Schedule',size=16); plt.show()

LR2 = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):

    print('#'*25)
    print(f'### Fold {i+1}')
    mix_gen = DataGenerator(train.iloc[train_index], shuffle=True, batch_size=32)
    train_gen = DataGenerator(train.iloc[train_index], shuffle=True, batch_size=32)
    valid_gen = DataGenerator(train.iloc[valid_index], shuffle=False, batch_size=64, mode='valid')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)

    # TRAIN MODEL
    model = chronoNet()
    checkpoint_callback = ModelCheckpoint(
            f'best_1D-Chrono_f{i}.h5',
            monitor='val_loss',  # Choose the metric to monitor for saving the best model
            save_best_only=True,  # Save only the best model
            mode='min',  # 'min' if monitoring loss, 'max' if monitoring accuracy
            verbose=1  # Display messages about the saving process
    )

    model.fit(train_gen, verbose=1,
              validation_data = valid_gen,
              epochs=30,
              callbacks=[checkpoint_callback],
                  )